In [ ]:
from utils.compression import compress_directories, decompress_directories

In [2]:
base_path = "data/gwilliams"
destination_path = "compressed_data/gwilliams"

compress_directories(
    base_path,
    destination_path,
    checksum_file_name="checksums.txt",
    compression_level=12,
    num_workers='mac'
)

Compressing directories: 100%|██████████| 28/28 [14:06<00:00, 30.22s/it]

Copied .DS_Store to compressed_data/gwilliams
Copied dataset_description.json to compressed_data/gwilliams
Copied README.txt to compressed_data/gwilliams
Copied participants.json to compressed_data/gwilliams
Copied participants.tsv to compressed_data/gwilliams
Compression complete.


In [ ]:
# base_path = "data/schoffelen"
# destination_path = "compressed_data/schoffelen"

# compress_directories(
#     base_path,
#     destination_path,
#     checksum_file_name="checksums.txt",
#     compression_level=12,
#     num_workers='mac'
# )

In [3]:
base_path = "compressed_data/gwilliams"
destination_path = "decompressed_data/gwilliams"

decompress_directories(
    base_path,
    destination_path,
    checksum_file_name="checksums.txt",
    num_workers='mac'
)

Decompressing files: 100%|██████████| 28/28 [03:58<00:00,  8.51s/it]

Copied .DS_Store to decompressed_data/gwilliams
Copied dataset_description.json to decompressed_data/gwilliams
Copied README.txt to decompressed_data/gwilliams
Copied participants.json to decompressed_data/gwilliams
Copied participants.tsv to decompressed_data/gwilliams
Decompression and verification complete
All files decompressed successfully.


In [5]:
base_path = "downloaded_data/gwilliams"
destination_path = "decompressed_data/gwilliams"

decompress_directories(
    base_path,
    destination_path,
    checksum_file_name="checksums.txt",
    num_workers='mac'
)

Decompressing files: 100%|██████████| 28/28 [04:02<00:00,  8.68s/it]

Copied .DS_Store to decompressed_data/gwilliams
Copied dataset_description.json to decompressed_data/gwilliams
Copied README.txt to decompressed_data/gwilliams
Copied participants.json to decompressed_data/gwilliams
Copied participants.tsv to decompressed_data/gwilliams
Decompression and verification complete
All files decompressed successfully.
